In [1]:
# Choose one of the matplotlib initialization for interactive or non-interactive
%matplotlib qt5
#import matplotlib
#matplotlib.use('Qt5Agg')

In [2]:
import sqlite3
import astropy.units as u
from astropy.coordinates import SkyCoord

import os
import numpy as np
import pandas as pd
import sqlite3
import lsst.daf.persistence as dafPersist
import lsst.geom
from astropy.visualization import (ZScaleInterval, AsinhStretch, ImageNormalize)
import astropy.units as u
from astropy.table import Table
import matplotlib.pyplot as plt

In [5]:
cwpRepo = '/home/gkovacs/data/repo_DM-17825/ingested/rerun/proc_2019-02-21'
cwpTemplateRepo = '/home/gkovacs/data/repo_DM-17825/templates'
my_dbName = '/home/gkovacs/data/repo_DM-17825/ingested/rerun/proc_2019-02-21/association.db'
mrawls_dbName = '/home/gkovacs/data/repo_DM-17825/mrawls_cw_processed2/association.db'
mrawls_repo = '/home/gkovacs/data/repo_DM-17825/mrawls_cw_processed2'

butlerCwp = dafPersist.Butler(cwpRepo)
butlerCwpTemplate = dafPersist.Butler(cwpTemplateRepo)
conn = sqlite3.connect(my_dbName)

# ==========

In [6]:
srcTable = Table.from_pandas(pd.read_sql_query('select * from  DiaSource '
                                               'where decl < -5.6 and decl > -5.8'
            ' and ra > 155.2 and ra < 155.3 and flags = 0', conn))


S = SkyCoord(srcTable['ra']*u.deg, srcTable['decl']*u.deg)

In [7]:
# Meredith's ppdb

m_conn = sqlite3.connect(mrawls_dbName)
m_srcTable = Table.from_pandas(pd.read_sql_query(
    'select * from DiaSource where decl < -5.6 and decl > -5.8'
            ' and ra > 155.2 and ra < 155.3 and flags = 0', m_conn))
m_S = SkyCoord(m_srcTable['ra']*u.deg, m_srcTable['decl']*u.deg)

In [8]:
print(pd.read_sql('select count(*) from DiaSource where decl < -5.6 and decl > -5.8'
            ' and ra > 155.2 and ra < 155.3',m_conn))

print(pd.read_sql('select count(*) from DiaSource where decl < -5.6 and decl > -5.8'
            ' and ra > 155.2 and ra < 155.3 and flags = 0',m_conn))

   count(*)
0      3486
   count(*)
0       678


In [9]:
print(pd.read_sql('select count(*) from DiaSource where decl < -5.6 and decl > -5.8'
            ' and ra > 155.2 and ra < 155.3',conn))

print(pd.read_sql('select count(*) from DiaSource where decl < -5.6 and decl > -5.8'
            ' and ra > 155.2 and ra < 155.3 and flags = 0',conn))

   count(*)
0      2697
   count(*)
0      1202


In [72]:
%run -n -i "plot_diasources_comparison.py"

In [73]:
plot_diasources_diffim(cwpRepo,mrawls_repo,srcTable,m_srcTable,Nmaxpanels=12)

1
2
3


Plotting only selected panels

In [89]:
%run -n -i "plot_diasources_comparison.py"

In [90]:
srcTable = Table.from_pandas(pd.read_sql_query('select * from  DiaSource '
                                               'where decl < -5.6 and decl > -5.8'
            ' and ra > 155.2 and ra < 155.3', conn))
m_srcTable = Table.from_pandas(pd.read_sql_query(
    'select * from DiaSource where decl < -5.6 and decl > -5.8'
            ' and ra > 155.2 and ra < 155.3', m_conn))

In [91]:
plot_diasources_diffim(cwpRepo,mrawls_repo,srcTable,m_srcTable,Nmaxpanels=12,selectPanels=[74,])

1


# ============

In [ ]:
%run -n -i "plot_calexp_template_diffim.py"

In [ ]:
patchList = ['10,8', '11,8', '12,8', '13,8',
             '10,7', '11,7', '12,7', '13,7',
             '10,9', '11,9', '12,9', '13,9',
             '10,5', '11,5', '12,5', '13,5',
             '10,6', '11,6', '12,6', '13,6',
             '10,10', '11,10', '12,10', '13,10']

In [ ]:
cwpObjTable = loadAllPpdbObjects(cwpRepo)
cwpMiniRegion = defMiniRegion(cwpObjTable)
cwpMiniUnflagged = cwpMiniRegion & (cwpObjTable['flags'] == 0)

In [12]:
min(5,6)

5

In [25]:
fig = plt.gcf()

In [26]:
ax = fig.gca()

In [31]:
ax.xaxis.get_major_locator()()

array([ 500., 1000., 1500., 2000.])

In [36]:
ax.xaxis.get_major_formatter()

In [37]:
ax2 = ax.twiny()

In [38]:
ax2.set_xlim(0,100)

(0, 100)

In [39]:
ax3 = ax.twinx()

In [40]:
ax3.set_ylim(-500,500)

(-500, 500)

Traceback (most recent call last):
  File "/ssd/gkovacs/lsstsw/miniconda/envs/lsst-scipipe/lib/python3.6/site-packages/matplotlib/backends/backend_qt5.py", line 519, in _draw_idle
    self.draw()
  File "/ssd/gkovacs/lsstsw/miniconda/envs/lsst-scipipe/lib/python3.6/site-packages/matplotlib/backends/backend_agg.py", line 433, in draw
    self.figure.draw(self.renderer)
  File "/ssd/gkovacs/lsstsw/miniconda/envs/lsst-scipipe/lib/python3.6/site-packages/matplotlib/artist.py", line 55, in draw_wrapper
    return draw(artist, renderer, *args, **kwargs)
  File "/ssd/gkovacs/lsstsw/miniconda/envs/lsst-scipipe/lib/python3.6/site-packages/matplotlib/figure.py", line 1475, in draw
    renderer, self, artists, self.suppressComposite)
  File "/ssd/gkovacs/lsstsw/miniconda/envs/lsst-scipipe/lib/python3.6/site-packages/matplotlib/image.py", line 141, in _draw_list_compositing_images
    a.draw(renderer)
  File "/ssd/gkovacs/lsstsw/miniconda/envs/lsst-scipipe/lib/python3.6/site-packages/matplotlib/ar

In [65]:
fig = plt.figure()

In [66]:
ax = fig.add_subplot(1,1,1)

In [67]:
ax.plot ([1,2],[3,4])

In [68]:
ax2 = ax.twinx()

In [69]:
ax2.set_ylim(0,100)

(0, 100)

In [70]:
ax3 = ax.twiny()

In [71]:
ax3.set_xlim(-10,10)

(-10, 10)